## Interuption DAG Task

...

In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

In [2]:
card_X = 2
card_Y = 2
card_Z = 2


observable_probs = np.arange(card_X * card_Y * card_Z)
observable_probs = observable_probs / observable_probs.sum()
observable_probs = observable_probs.reshape(card_X, card_Y, card_Z)

dist_XYZ = {}
# defining a p distribution based on cardinality of A, B, C
for x, y, z in itertools.product(range(card_X), range(card_Y), range(card_Z)):
    prob = observable_probs[x, y, z]
    dist_XYZ[(x, y, z)] = prob

In [3]:
# given distribution P_XYZ get P_XY|Z:
dist_XY_giv_Z = {}
for x, y, z in itertools.product(range(card_X), range(card_Y), range(card_Z)):
    prob = dist_XYZ[(x, y, z)]
    dist_XY_giv_Z[(x, y, z)] = prob / observable_probs[:, :, z].sum()

distribution feasible?

In [4]:
dist_XYZ

{(0, 0, 0): 0.0,
 (0, 0, 1): 0.03571428571428571,
 (0, 1, 0): 0.07142857142857142,
 (0, 1, 1): 0.10714285714285714,
 (1, 0, 0): 0.14285714285714285,
 (1, 0, 1): 0.17857142857142858,
 (1, 1, 0): 0.21428571428571427,
 (1, 1, 1): 0.25}

In [5]:
print(sum(dist_XYZ.values()) == 1)

True


In [6]:
dist_XY_giv_Z

{(0, 0, 0): 0.0,
 (0, 0, 1): 0.0625,
 (0, 1, 0): 0.16666666666666666,
 (0, 1, 1): 0.1875,
 (1, 0, 0): 0.3333333333333333,
 (1, 0, 1): 0.3125,
 (1, 1, 0): 0.5,
 (1, 1, 1): 0.4375}

In [7]:
m = gp.Model()
#m.Params.LogToConsole = 0

# P_Z(z) = sum_{X,Y} P_XYZ(x,y,z)
def P_Z(z):
    return sum([dist_XYZ[(X, Y, z)] for X in range(card_X) for Y in range(card_Y)])


P_Xb_Y_giv_Z_Xs = m.addMVar(shape = (card_X, card_Y, card_Z, card_X), vtype=GRB.CONTINUOUS, name="P_Xb_Y_giv_Z_Xs", lb=0, ub=1)
P_Xb_giv_Z_Xs = sum([P_Xb_Y_giv_Z_Xs[:,y,:,:] for y in range(card_Y)]) # P_Xb_giv_Z_Xs[xb,z,xs]
P_Y_giv_Xs = sum([P_Xb_Y_giv_Z_Xs[xb,:,z,:]*P_Z(z) for xb in range(card_X)]) # P_Y_giv_Z_Xs[y,z,x] == P_Y_giv_Xs[y,x]
P_Y_giv_Z_Xs = sum([P_Xb_Y_giv_Z_Xs[xb,:,:,:] for xb in range(card_X)]) # P_Y_giv_Z_Xs[y,z,xs]


# consistency constraints
for x, y, z in itertools.product(range(card_X), range(card_Y), range(card_Z)):
    m.addConstr(P_Xb_Y_giv_Z_Xs[x,y,z,x] == dist_XY_giv_Z[(x, y, z)])
    P_Y_Z_giv_Xs = P_Y_giv_Xs[y,x]*P_Z(z)

for xb, xs, z in itertools.product(range(card_X), range(card_X), range(card_Z)):
    m.addConstr(P_Xb_giv_Z_Xs[xb, z, 0] == P_Xb_giv_Z_Xs[xb, z, 1])

# for x,y,z in itertools.product(range(card_X), range(card_Y), range(card_Z)):
#     P_Y_Z_giv_Xs = P_Y_giv_Xs[y,x]*P_Z(z)

for x,y in itertools.product(range(card_X), range(card_Y)):
    m.addConstr(P_Y_giv_Z_Xs[y,0,x] == P_Y_giv_Z_Xs[y,1,x])

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [8]:
# [P_Xb_Y_giv_Z_Xs[:,y,:,:]] # the array is made up of all combinations with y fixed

In [9]:
# sum([P_Xb_Y_giv_Z_Xs[xb,:,:,:] for xb in range(card_X)]) 

In [10]:
# optomize for P_YdoX#
# P_YdoX# = P_Y_giv_Xs
def P_YdoXs(y,x):
    m.setObjective(P_Y_giv_Xs[y,x], GRB.MINIMIZE)
    m.optimize()
    min_val = P_Y_giv_Xs[y,x].getValue()

    m.setObjective(P_Y_giv_Xs[y,x], GRB.MAXIMIZE)
    m.optimize()
    max_val = P_Y_giv_Xs[y,x].getValue()

    print("\nmin value: ", min_val)
    print("max value: ", max_val)
    print("distance:", max_val - min_val)

In [11]:
P_YdoXs(1,1)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 16 columns and 56 nonzeros
Model fingerprint: 0xa8283788
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-02, 5e-01]
Presolve removed 20 rows and 16 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8571429e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.857142857e-01
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AV

In [12]:
#m.computeIIS()
#m.write("model.ilp")